# *k*-Medians

In this assignment we'll be comparing some different methods of finding *k* medians.

First, some boilerplate out of the way:

Note that we're using the kcenter.py module from the last assignment, and we have a new module called kmedian.py.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.utils import check_random_state
from sklearn.metrics import pairwise_distances, pairwise_distances_argmin_min
from sklearn.utils.extmath import row_norms
from sklearn.datasets import load_iris

from kmedian import KMedians
from kcenter import KCenters

import matplotlib.pyplot as plt
import seaborn as sns

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# Enable high resolution PNGs
%config InlineBackend.figure_formats = {'png', 'retina'}

# Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': '#DFDFE5'}
sns.set(context='notebook', style='darkgrid', rc=rc)

## Plotting

We'll be plotting a few things in this assignment, so let's make a function that takes care of this for us. It takes a dataframe, `df`, as well as a list of features `vars` that we care about.

The DataFrame `df` should have a column `ypred` that contains integers representing the cluster assignment.

This function will do a "pairplot," which makes a set of plots that pairs up every feature listed in `vars`, and produces a 2-d scatterplot. We can add cluster labels as hue, and this function takes care of that.

In [ ]:
def plot_clusters(df, vars=None): 
    # Plot the cluster labels:
    g = sns.pairplot(data=df, hue='ypred', palette='Dark2', vars=vars)
    
    plt.show()

## Dataset 

Let's load the iris dataset, and massage it for our needs. Almost every dataset needs some kind of treatment like this.

In [ ]:

# First, load the iris data
iris = load_iris()

# The data is in list format, so make it into an array:
X_iris = np.array(iris.data)
iris_samples, iris_features = X_iris.shape

# It will be convenient later to randomize the order, 
# but it's less confusing to do it here
order = check_random_state(201609220).permutation(iris_samples)
X_iris = X_iris[order]

# Give the columns of the DataFrame the names of the features
df_iris = pd.DataFrame(X_iris, columns=iris['feature_names'])

# Add the actual labels from the dataset into the DataFrame 
# for comparison
df_iris['real_labels'] = np.array(iris['target'])[order]

# Finally, the data contains a description of the dataset,
# so let's print that out:
print iris['DESCR']

## Local search

The `kmedian.py` file contains code to perform the local search algorithm. Let's make a quick function that trains a set of cluster centers with local search. This will also save the labels in the DataFrame `df`, and will return the model object, which has properties `medoids_` and `cost_history_` that we'll use.

Note: this implementation takes much longer on larger datasets. We've set `verbose` to `True` so that you can see the evolution of the cluster centers:

In [ ]:
def local_search_with(X, df, n_clusters=3, metric='l2'):
    
    clust = KMedians(n_clusters=n_clusters, metric=metric, tol=1e-5, verbose=True, 
                      random_state=201609221)
    
    df['ypred'] = clust.fit_predict(X)
    
    return clust
    
ls_model = local_search_with(X_iris, df_iris, metric='l2')

### Plot the results of local search
Let's plot the clusters, along with cluster centers, and let's also take a look at how the costs have changed over time:

In [ ]:
plot_clusters(df_iris, 
              vars=['sepal length (cm)', 'sepal width (cm)', 
                    'petal length (cm)', 'petal width (cm)', 
                    'real_labels'])
plt.plot(ls_model.cost_history_)

## Problem

Let's implement a different algorithm for finding $k$ medians, the *$k$-medoids* algorithm. Unlike local search, this algorithm is very straightforward to implement.

First, given the current cluster labels, we're going to choose the correct medians for the job:

In [ ]:
def choose_medians(X, labels, n_clusters, metric='euclidean'):
    """
    X: float array, (n_samples, n_features) 
        The input data
    labels: int array, (n_samples,)
        Numbers that indicate the cluster that the corresponding row of X belongs to
    n_clusters:
        The number of unique elements in labels
    -----------------------------------------------
    returns:
    medians: float array, (n_clusters, n_features)
    """
    
    # Get dimensions of `X`
    # This information is contained inside the `X` variable. 
    # Use it to populate `n_samples` and `n_features`.
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Allocate the return variable
    medians = np.empty((n_clusters, n_features), dtype=np.float64)
    
    # For every cluster
    for i in range(n_clusters):
        # Construct a new array `Xi` that contains the examples with label i
        # Note: there might only be one, the cluster center. If this is the 
        #       case, then make sure that `Xi` has shape (1,n_features) and 
        #       not shape (n_features,).
        # YOUR CODE HERE
        raise NotImplementedError()

        # Find the best example from `Xi` to use as the median. That is, 
        # compute the row that has the smallest sum of distances to every 
        # other row of `Xi`.
        # 
        # To do this, take advantage of the `pairwise_distances` function
        # from sklearn.metric (it's been imported already). Note that 
        # `metric` has been passed to this function, so you can use that to
        # pass to the `metric` parameter of pairwise_distances.
        # 
        # Hint: This is much easier if you take advantage numpy array func-
        #       tionality.
        # YOUR CODE HERE
        raise NotImplementedError()

        # Put the row that you found in the previous step into the corres-
        # ponding row of `medians`:
        # YOUR CODE HERE
        raise NotImplementedError()
        
    return medians

In [ ]:
n_clusters = 9
X_test = np.eye(n_clusters)
labels = np.arange(n_clusters)

medians = choose_medians(X_test, labels, n_clusters, metric='l2')

assert(medians.shape == (n_clusters,n_clusters))
assert(np.isclose(medians.sum(axis=0), X_test.sum(axis=0)).all())

X_test = np.zeros((2000,n_clusters))
labels = check_random_state(201609222).randint(n_clusters, size=(X_test.shape[0],))
X_test[np.arange(2000),labels] = 1.0

medians = choose_medians(X_test, labels, n_clusters, metric='l2')

assert(medians.shape == (n_clusters,n_clusters))
assert(np.isclose(medians.sum(axis=0), np.ones((n_clusters,))).all())

## Main algorithm

We'll now use the function we just wrote in the $k$-medoids algorithm. We pass in our data, a set of inital centers, a tolerance parameter to tell us when to stop, and a metric:

In [ ]:
def k_medoids(X, init_centers, tol=1e-4, metric='l2'):
    n_clusters, n_features = init_centers.shape
    
    # Find the right labels for the examples against the initial centers
    labels, distances = pairwise_distances_argmin_min(X, init_centers, metric=metric)
    # Record the initial cost
    cost_history = [distances.sum()]

    # Keep going until we don't have a significant change (according to `tol`):
    while True:
        # Use our function that we just wrote
        centers = choose_medians(X, labels, n_clusters, metric=metric)
        # Get the new labels and distances
        labels, distances = pairwise_distances_argmin_min(X, centers, metric=metric)
        # Record the cost
        cost_history.append(distances.sum())
        # If we didn't improve significantly, stop
        if cost_history[-1] >= (1-tol)*cost_history[-2]:
            break
            
    return centers, labels, cost_history

## Running $k$-medoids

Let's take a look at our new algorithm. The performance of $k$-medoids depends greatly on the initial cluster centers we choose. Let's choose random centers first, the easiest way to select centers:

In [ ]:
# `X` was shuffled, so just take the first 3 rows as cluster centers
init_centers = X_iris[:3,:]

# Copy the DataFrame so we don't pollute previous results
df2 = df_iris.copy()
centers, df2['ypred'], cost_history_1 = k_medoids(X_iris, init_centers, tol=1e-5, metric='l2')

plot_clusters(df2, 
              vars=['sepal length (cm)', 'sepal width (cm)', 
                    'petal length (cm)', 'petal width (cm)', 
                    'real_labels'])

# Problem

How do the clusters compare to the results from local search? Based on the plot, which would you choose and why? The bottom right hand plot features a histogram that indicates the distribution of cluster labels against the real labels of the dataset. How does this compare to the first plot?

YOUR ANSWER HERE

## Comparing performance

Now let's compare the cost at each iteration:

In [ ]:
plt.plot(ls_model.cost_history_, label='Local Search')
plt.plot(cost_history_1, label='k-Medoids, random initial centers')
plt.legend()
plt.show()

## Problem

How do the algorithms compare now? Would you change your previous answer based on what you see? Why or why not?

YOUR ANSWER HERE

# $k$-Medoids, with $k$-Centers initial centers

Now let's try choosing centers by using the $k$-centers algorithm from the previous assignment:

In [ ]:
init_centers = KCenters(n_clusters=3, random_state=201609223).fit(X_iris).cluster_centers_

df3 = df_iris.copy()
centers, df3['ypred'], cost_history_2 = k_medoids(X_iris, init_centers, tol=1e-5, metric='l2')

plot_clusters(df3, 
              vars=['sepal length (cm)', 'sepal width (cm)', 
                    'petal length (cm)', 'petal width (cm)', 
                    'real_labels'])

# Problem

How do the clusters compare to the results from local search and k-medoids with random initial clusters? Again, based on the plot, which would you choose and why?

YOUR ANSWER HERE

## Compare performance

In [ ]:
plt.plot(ls_model.cost_history_, label='Local Search')
plt.plot(cost_history_1, label='k-Medoids, random initial centers')
plt.plot(cost_history_2, label='k-Medoids, k-Centers initial centers')
plt.legend()
plt.show()

## Problem

Again, how do the algorithms compare? Would you change your previous answer based on what you see? Why or why not?

YOUR ANSWER HERE